### add "fraudulent" = 0 for original real data

In [1]:
import pandas as pd


df = pd.read_csv("Jadarat_data.csv", encoding="utf-8")


if "fraudulent" not in df.columns:
    df.insert(0, "fraudulent", 0)
else:
    df["fraudulent"] = 0

df.head()

,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
0,0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة مقر العالم للسفريات,1-317262,خاص,متوسطة فئة أ,أنشطة وكالات السياحة والسفر,"['Language data', 'اللغة الانجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350419,0 Years,both
1,0,بائع,27/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة عالم الكهرباء للمقاولات,4-1324428,خاص,متوسطة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary', '5000.0']",دوام كامل,0 / 3,20202026350389,0 Years,both
2,0,أخصائي عمليات موارد بشرية,27/05/1444,['تنفيذ الإجراءات والأنظمة والنماذج الخاصة بمر...,[' تنفيذ الإجراءات والأنظمة والنماذج الخاصة ...,شركه دار السلام,1-155294,خاص,متوسطة فئة أ,ترميمات المباني السكنية والغير سكنية,"['Language data', 'الانجليزيه', 'محترف']",الرياض,ATH THUMA...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026350347,2 Years,both
3,0,ميكانيكي سيارات,27/05/1444,['تشخيص أعطال السيارات وإصلاحها وتنفيذ برامج ا...,[' فحص أداء المعدّات الكهربائية والميكانيكية...,مؤسسة لمكو لغيار الزيوت,8-1925495,خاص,صغيرة فئة ب,NaN,"['Skill data', 'صيانة السيارات وتقييم الاعطال'...",المنطقة الشرقية,AL HUFUF...,"['Salary', '5000.0']",دوام كامل,0 / 10,20202026350219,0 Years,M
4,0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة فكرة اليمامة للمقاولات,1-2356639,خاص,كبيرة,الإنشاءات العامة للمباني السكنية,"['Skill data', 'تحمل ضغط العمل', 'محترف', 'Lan...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350043,0 Years,both


### Merge Fake with Real data

In [2]:
import pandas as pd
import glob
import os


main_csv = "Jadarat_data.csv"
main_df = pd.read_csv(main_csv, encoding="utf-8-sig")


if "fraudulent" not in main_df.columns:
    main_df["fraudulent"] = 0


json_files = glob.glob("FakeData/*.json")

fake_dfs = []
for jf in json_files:
    try:
        fake_df = pd.read_json(jf, encoding="utf-8")
        if "fraudulent" not in fake_df.columns:
            fake_df["fraudulent"] = 1
        fake_dfs.append(fake_df)
        print(f"Loaded JSON: {jf} -> {fake_df.shape}")
    except Exception as e:
        print(f"Skipped {jf}: {e}")

if not fake_dfs:
    raise RuntimeError("لم يتم العثور على ملفات JSON صالحة في المجلد FakeData.")


preferred_cols = [
    "fraudulent","job_title","job_date","job_desc","job_tasks",
    "comp_name","comp_no","comp_type","comp_size","eco_activity",
    "qualif","region","city","benefits","contract","positions",
    "job_post_id","exper"
]

all_fake = pd.concat(fake_dfs, ignore_index=True)
all_cols_union = list(dict.fromkeys(preferred_cols + list(main_df.columns) + list(all_fake.columns)))

main_aligned = main_df.reindex(columns=all_cols_union)
fake_aligned = all_fake.reindex(columns=all_cols_union)


combined_df = pd.concat([main_aligned, fake_aligned], ignore_index=True)
rest_cols = [c for c in combined_df.columns if c not in preferred_cols]
combined_df = combined_df[preferred_cols + rest_cols]
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)


out_csv = "mergedFakeWithRealData.csv"
combined_df.to_csv(out_csv, index=False, encoding="utf-8-sig")


fraud_count = (combined_df["fraudulent"] == 1).sum() if "fraudulent" in combined_df.columns else 0
real_count  = (combined_df["fraudulent"] == 0).sum() if "fraudulent" in combined_df.columns else 0

print("Merge done.")
print(f"Final shape: {combined_df.shape}")
print(f"Legitimate rows (fraudulent=0): {real_count}")
print(f"Fake rows (fraudulent=1): {fraud_count}")
print(f"Saved to: {os.path.abspath(out_csv)}")


Loaded JSON: FakeData\benefits_contract_mismatch.json -> (50, 18)
Loaded JSON: FakeData\compsize_benefits_mismatch.json -> (50, 18)
Loaded JSON: FakeData\contract_jobtasks_mismatch.json -> (50, 18)
Loaded JSON: FakeData\ecoactivity_jobtasks_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobdate_jobdesc_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobdesc_gender_mismatch.json -> (50, 17)
Loaded JSON: FakeData\jobtasks_exper_mismatch.json -> (50, 16)
Loaded JSON: FakeData\jobtitle_comptype_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_exper_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_jobdesc_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_qualif_mismatch.json -> (50, 18)
Loaded JSON: FakeData\qualif_jobtitle_mismatch.json -> (50, 16)
Loaded JSON: FakeData\region_city_mismatch.json -> (50, 18)
Merge done.
Final shape: (2120, 20)
Legitimate rows (fraudulent=0): 1470
Fake rows (fraudulent=1): 650
Saved to: c:\Users\Lolo-\Downloads\machine-learning-projec

### To double check if Fake is exist

In [3]:
# extract only Fake data
fraud_df = combined_df[combined_df["fraudulent"] == 1]


import IPython.display as disp
disp.display(fraud_df)


print("Fake rows number", len(fraud_df))


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender,type
3,1,محاسب,27/05/1444,['مطلوب فورا - للتقديم عبر الواتساب.'],"[' إدخال بيانات بسيطة', ' الرد على رسائل و...",شركة الثروة السريعة,3-622795,غير ربحي,صغيرة فئة أ,NaN,NaN,المدينة المنورة,AR RIYADH...,"['Salary','30000']",دوام كامل,0 / 17,FRAUD2025CONTASK1036,3 Years,NaN,NaN
4,1,محاسب,27/05/1444,['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...,"[' توزيع بروشورات في الشوارع', ' مكالمات و...",شركة المال السهل,5-973282,غير ربحي,متوسطة فئة أ,NaN,NaN,المدينة المنورة,AR RIYADH...,"['Salary','15000']",عقد مؤقت,0 / 26,FRAUD20251017,0 Years,NaN,NaN
5,1,مترجم,29/05/1444,['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...,[' لا يوجد مهام محددة، فقط إرسال سيرة ذاتية'...,شركة المال السهل,5-274206,خاص,صغيرة فئة أ,NaN,NaN,مكة المكرمة,JEDDAH...,"['Salary','20000']",دوام جزئي,0 / 13,FRAUD20251036,1 Years,NaN,NaN
7,1,طبيب,15/05/1440,['فرصة محدودة - التحق فوراً قبل إغلاق التقديم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",شركة المال السهل,7-424052,حكومي سري,كبيرة,NaN,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary','30000']",دوام جزئي,0 / 12,FRAUD2025DATEDESC1019,1 Years,NaN,NaN
13,1,سكرتير,27/05/1444,['راتب عالي جدا بدون أي متطلبات إضافية.'],"[' وضع استراتيجيات طويلة الأمد', ' تطوير خ...",مؤسسة النجاح الفوري,2-604580,غير ربحي,متوسطة فئة أ,NaN,NaN,الرياض,AL MADINAH...,"['Salary','15000']",دوام جزئي,0 / 29,FRAUD2025CONTASK1014,1 Years,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098,1,مراسل,30/05/1444,['فرصة مميزة برواتب تنافسية.'],"[' مهام بسيطة وروتينية', ' ترتيب ملفات', '...",مركز أفق الأعمال,NaN,غير ربحي,كبيرة,NaN,PhD,المنطقة الشرقية,TABUK...,"['Salary','8000']",عقد مؤقت,NaN,NaN,0 Years,NaN,Fake
2107,1,مسوق إلكتروني,28/05/1444,['مطلوب فورا - للتقديم عبر الواتساب.'],"[' إعداد تقارير', ' متابعة البريد', ' ال...",جهة حكومية سرية,8-599168,غير ربحي,متوسطة فئة أ,NaN,NaN,الرياض,AL MADINAH...,"['Salary','60000']",عقد مؤقت,0 / 20,FRAUD2025BENCONT1045,3 Years,NaN,NaN
2109,1,خدمة عملاء,20/02/1439,['مطلوب فورا! آخر موعد للتقديم اليوم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",مؤسسة النجاح الفوري,2-268270,خاص,متوسطة فئة أ,NaN,NaN,المنطقة الشرقية,JEDDAH...,"['Salary','25000']",دوام جزئي,0 / 21,FRAUD2025DATEDESC1015,3 Years,NaN,NaN
2111,1,مسوق إلكتروني,29/05/1444,['وظيفة عاجلة - تواصل عبر البريد الشخصي.'],"[' إعداد تقارير', ' متابعة البريد', ' ال...",شركة الثروة السريعة,1-752092,خاص,صغيرة فئة أ,NaN,NaN,مكة المكرمة,AL MADINAH...,"['Salary','30000']",دوام جزئي,0 / 40,FRAUD2025BENCONT1012,1 Years,NaN,NaN


Fake rows number 650


### Check of merge 5 fake - 5 real

In [4]:
import pandas as pd

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")

# 
fake_head = df[df["fraudulent"] == 1].head(5).copy()
fake_head["Type"] = "Fake"

real_head = df[df["fraudulent"] == 0].head(5).copy()
real_head["Type"] = "Real"

sample_df = pd.concat([fake_head, real_head], ignore_index=True)

# display no save
sample_df


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,...,region,city,benefits,contract,positions,job_post_id,exper,gender,type,Type
0,1,محاسب,27/05/1444,['مطلوب فورا - للتقديم عبر الواتساب.'],"[' إدخال بيانات بسيطة', ' الرد على رسائل و...",شركة الثروة السريعة,3-622795,غير ربحي,صغيرة فئة أ,NaN,...,المدينة المنورة,AR RIYADH...,"['Salary','30000']",دوام كامل,0 / 17,FRAUD2025CONTASK1036,3 Years,NaN,NaN,Fake
1,1,محاسب,27/05/1444,['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...,"[' توزيع بروشورات في الشوارع', ' مكالمات و...",شركة المال السهل,5-973282,غير ربحي,متوسطة فئة أ,NaN,...,المدينة المنورة,AR RIYADH...,"['Salary','15000']",عقد مؤقت,0 / 26,FRAUD20251017,0 Years,NaN,NaN,Fake
2,1,مترجم,29/05/1444,['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...,[' لا يوجد مهام محددة، فقط إرسال سيرة ذاتية'...,شركة المال السهل,5-274206,خاص,صغيرة فئة أ,NaN,...,مكة المكرمة,JEDDAH...,"['Salary','20000']",دوام جزئي,0 / 13,FRAUD20251036,1 Years,NaN,NaN,Fake
3,1,طبيب,15/05/1440,['فرصة محدودة - التحق فوراً قبل إغلاق التقديم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",شركة المال السهل,7-424052,حكومي سري,كبيرة,NaN,...,المنطقة الشرقية,AD DAMMAM...,"['Salary','30000']",دوام جزئي,0 / 12,FRAUD2025DATEDESC1019,1 Years,NaN,NaN,Fake
4,1,سكرتير,27/05/1444,['راتب عالي جدا بدون أي متطلبات إضافية.'],"[' وضع استراتيجيات طويلة الأمد', ' تطوير خ...",مؤسسة النجاح الفوري,2-604580,غير ربحي,متوسطة فئة أ,NaN,...,الرياض,AL MADINAH...,"['Salary','15000']",دوام جزئي,0 / 29,FRAUD2025CONTASK1014,1 Years,NaN,NaN,Fake
5,0,بائع,28/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",مؤسسة قيادة المباني للمقاولات,1-2524485,خاص,صغيرة فئة ب,NaN,...,الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 6,20202026352624,0 Years,both,NaN,Real
6,0,كيميائي,29/06/1444,['إجراء الأبحاث والتحاليل الكيميائية للعناصر و...,[' إجراء الأبحاث والتجارب والتحاليل الكمية و...,دار القياس للتجاره فرع شركة ديارات التجاريه,1-1771214,خاص,صغيرة فئة ب,إصلاح وصيانة معدات القياس والاختبار والمراقبة ...,...,الرياض,AR RIYADH...,"['Salary', '6500.0']",دوام كامل,0 / 4,20202026406972,4 Years,M,NaN,Real
7,0,محاسب,21/04/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة الأندلس للشقق الفندقية,9-1984450,خاص,صغيرة فئة ب,أنشطة إدارة العقارات مقابل عمولة,...,مكة المكرمة,JEDDAH...,"['Salary', '6000.0']",دوام كامل,0 / 1,20202026271430,2 Years,both,NaN,Real
8,0,بائع,02/06/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",مؤسسة أيوب سعد بن سعيد ال غوينم القحطاني التجارية,11-1953774,خاص,صغيرة فئة ب,البيع بالجملة والتجزئة لقطع الغيار الجديده للس...,...,عسير,ABRAQ AN ...,"['Salary', '4500.0']",دوام كامل,0 / 1,20202026357136,0 Years,M,NaN,Real
9,0,أخصائي تسويق,05/04/1444,['تحديد الأهداف وتطوير السياسات التسويقية للسل...,[' تحديد الأهداف العامة والفرعية للتسويق وتط...,مجموعة البارقة للتشغيل والصيانة,15-1950752,خاص,صغيرة فئة ب,تقديم موارد بشرية أخرى,...,الرياض,AR RIYADH...,"['Salary', '6000.0', 'Benefits', 'تحدد بعد الم...",دوام كامل,0 / 4,20202026228880,2 Years,both,NaN,Real


### ADD "type" colmn

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")


if "fraudulent" not in df.columns:
    raise ValueError("عمود 'fraudulent' غير موجود في الملف.")


df["Type"] = np.where(df["fraudulent"] == 1, "Fake",
              np.where(df["fraudulent"] == 0, "Real", "Unknown"))


df.to_csv("mergedFakeWithRealData.csv", index=False, encoding="utf-8-sig")


df.head()


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,...,region,city,benefits,contract,positions,job_post_id,exper,gender,type,Type
0,0,بائع,28/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",مؤسسة قيادة المباني للمقاولات,1-2524485,خاص,صغيرة فئة ب,NaN,...,الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 6,20202026352624,0 Years,both,NaN,Real
1,0,كيميائي,29/06/1444,['إجراء الأبحاث والتحاليل الكيميائية للعناصر و...,[' إجراء الأبحاث والتجارب والتحاليل الكمية و...,دار القياس للتجاره فرع شركة ديارات التجاريه,1-1771214,خاص,صغيرة فئة ب,إصلاح وصيانة معدات القياس والاختبار والمراقبة ...,...,الرياض,AR RIYADH...,"['Salary', '6500.0']",دوام كامل,0 / 4,20202026406972,4 Years,M,NaN,Real
2,0,محاسب,21/04/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة الأندلس للشقق الفندقية,9-1984450,خاص,صغيرة فئة ب,أنشطة إدارة العقارات مقابل عمولة,...,مكة المكرمة,JEDDAH...,"['Salary', '6000.0']",دوام كامل,0 / 1,20202026271430,2 Years,both,NaN,Real
3,1,محاسب,27/05/1444,['مطلوب فورا - للتقديم عبر الواتساب.'],"[' إدخال بيانات بسيطة', ' الرد على رسائل و...",شركة الثروة السريعة,3-622795,غير ربحي,صغيرة فئة أ,NaN,...,المدينة المنورة,AR RIYADH...,"['Salary','30000']",دوام كامل,0 / 17,FRAUD2025CONTASK1036,3 Years,NaN,NaN,Fake
4,1,محاسب,27/05/1444,['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...,"[' توزيع بروشورات في الشوارع', ' مكالمات و...",شركة المال السهل,5-973282,غير ربحي,متوسطة فئة أ,NaN,...,المدينة المنورة,AR RIYADH...,"['Salary','15000']",عقد مؤقت,0 / 26,FRAUD20251017,0 Years,NaN,NaN,Fake


### drop "job_post_id", "positions", "comp_no"

In [6]:
import pandas as pd

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")

df = df.drop(columns=["job_post_id", "positions", "comp_no"])

print(df.shape)
print(df.head())

# update to merged CSV
df.to_csv("mergedFakeWithRealData.csv", index=False, encoding="utf-8-sig")


(2120, 18)
   fraudulent job_title    job_date  \
0           0      بائع  28/05/1444   
1           0   كيميائي  29/06/1444   
2           0     محاسب  21/04/1444   
3           1     محاسب  27/05/1444   
4           1     محاسب  27/05/1444   

                                            job_desc  \
0  ['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...   
1  ['إجراء الأبحاث والتحاليل الكيميائية للعناصر و...   
2  ['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...   
3             ['مطلوب فورا - للتقديم عبر الواتساب.']   
4  ['مطلوب انضمام فوري - راتب ٤٠٠٠٠ ريال مع عمولا...   

                                           job_tasks  \
0  ['   بيع مجموعة من السلع والخدمات للعملاء.', '...   
1  ['   إجراء الأبحاث والتجارب والتحاليل الكمية و...   
2  ['   تدقيق المطالبات المالية والتأكد من اكتمال...   
3  ['   إدخال بيانات بسيطة', '   الرد على رسائل و...   
4  ['   توزيع بروشورات في الشوارع', '   مكالمات و...   

                                     comp_name comp_type     comp_size  \
0     